---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [39]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [2]:
import numpy as np
import pandas as pd

In [2]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ],
    columns=["State", "RegionName"]  )
    The following cleaning needs to be done:
    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    df = pd.DataFrame([], columns=['State', 'RegionName'])
    with open('university_towns.txt', 'r') as f:
        state = ""
        for line in f:
            if '[edit]' in line:
                state = line[:line.find('[')].strip()
                continue
            region = line.strip()
            if '(' in region:
                region = region[:(region.find('(') - 1)]
            df = df.append(pd.DataFrame([[state, region]], columns=['State', 'RegionName']), ignore_index=True)
    return df
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [67]:
# def get_list_of_university_towns():
#     '''Returns a DataFrame of towns and the states they are in from the 
#     university_towns.txt list. The format of the DataFrame should be:
#     DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
#     columns=["State", "RegionName"]  )
    
#     The following cleaning needs to be done:

#     1. For "State", removing characters from "[" to the end.
#     2. For "RegionName", when applicable, removing every character from " (" to the end.
#     3. Depending on how you read the data, you may need to remove newline character '\n'. '''
#     with open('university_towns.txt','r') as p:
#         data=p.read()
#     dict1={}
#     list1=[]
#     for d in data.split("\n"):
#         if d.find("[edit]")!=-1:
#             index=d.find("[")
#             temp_k=d[0:index]
#             list1=[]
#         else:
#             ind=d.find('(')
#             tem=d[0:ind]
#             tem=tem.rstrip()
#             list1.append(tem)
#             dict1[temp_k]=list1
#     list2=[]
#     for k,v in dict1.items():
#         len1=len(v)
#         for i in range(len1):
#             list2.append(k)
#             i=i+1
#     list3=[]
#     for v in dict1.values():
#         for x in v:
#             list3.append(x)
#     df=pd.DataFrame([list2,list3]).T
#     df.columns=['State','RegionName']
#     df1=df.iloc[:len(df)-1,]
#     return df1


In [3]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    test=pd.read_excel('gdplev.xls',skiprows=7,names=['Year','GDP_Current','GDP_2009','NR','Year_Quat','GDP_Current_Quat','GDP_2009_Quat','idk'])
    test1=test.iloc[212:,4:7]
    test1.set_index(['Year_Quat'],inplace=True)
    test1.drop(['GDP_Current_Quat'],inplace=True,axis=1)
    for i in range(len(test1)-2):
        if (test1.iloc[i]['GDP_2009_Quat']<test1.iloc[i-1]['GDP_2009_Quat']) and (test1.iloc[i+1]['GDP_2009_Quat']<test1.iloc[i]['GDP_2009_Quat']):
            return test1.iloc[i].name
    return None
get_recession_start()

'2008q3'

In [4]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    test=pd.read_excel('gdplev.xls',skiprows=7,names=['Year','GDP_Current','GDP_2009','NR','Year_Quat','GDP_Current_Quat','GDP_2009_Quat','idk'])
    test1=test.iloc[212:,4:7]
    test1.set_index(['Year_Quat'],inplace=True)
    test1.drop(['GDP_Current_Quat'],inplace=True,axis=1)
    val=get_recession_start()
    test2=test1.loc[val:,]
    for i in range(len(test2)-2):
        if (test2.iloc[i]['GDP_2009_Quat']>test2.iloc[i-1]['GDP_2009_Quat']) and (test2.iloc[i-1]['GDP_2009_Quat']>test2.iloc[i-2]['GDP_2009_Quat']):
            return test2.iloc[i].name
    return None
get_recession_end()

'2009q4'

In [5]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    test=pd.read_excel('gdplev.xls',skiprows=7,names=['Year','GDP_Current','GDP_2009','NR','Year_Quat','GDP_Current_Quat','GDP_2009_Quat','idk'])
    test1=test.iloc[212:,4:7]
    test1.set_index(['Year_Quat'],inplace=True)
    test1.drop(['GDP_Current_Quat'],inplace=True,axis=1)
    start=get_recession_start()
    end=get_recession_end()
    test2=test1.loc[start:end,]
    return test2['GDP_2009_Quat'].idxmin()
get_recession_bottom()

'2009q2'

In [7]:
def get_quater(y,m):
    if m<=3:
        quater=1
    elif m<=6:
        quater=2
    elif m<=9:
        quater=3
    elif m<=12:
        quater=4
    return (str(y)+'q'+str(quater))

In [45]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    list1=[]
    test=pd.read_csv('City_Zhvi_AllHomes.csv')
    test1=test.drop(['RegionID','Metro','CountyName','SizeRank'],axis=1)
    states_dict = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    for val in test1['State']:
        if val in states_dict:
            list1.append(states_dict[val])
        else:
            list1.append('NotFound')
    test1['State_New']=list1
    test1.drop(['State'],axis=1,inplace=True)
    test1.rename(columns={'State_New':'State'},inplace=True)
    test2=test1.set_index(['State','RegionName'])
    test3=test2.iloc[:,45:]
    l=len(test3.columns)
    i=0
    while i<l:
        col_name = test3.iloc[:, i].name
        year=int(col_name.split('-')[0])
        month=int(col_name.split('-')[1])
        quater=get_quater(year,month)
        test3[quater]=test3.iloc[:,i:i+3].mean(axis=1)
        i+=3
    test4=test3.iloc[:,l:]
    return test4
convert_housing_data_to_quarters()

In [48]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    #price_ratio=quarter_before_recession/recession_bottom
    from scipy.stats import ttest_ind
    orig_df=convert_housing_data_to_quarters()
    start=orig_df.columns.get_loc(get_recession_start())
    bottom=orig_df.columns.get_loc(get_recession_bottom())
    #newdf=orig_df.iloc[:,start:bottom+1]
    orig_df['price_ratio']=orig_df.iloc[:,start-1]/orig_df.iloc[:,bottom]
    newdf=pd.DataFrame(orig_df.loc[:,'price_ratio'])
    newdf.reset_index(inplace=True)
    ut=get_list_of_university_towns()
    
    univ_prices=pd.merge(ut,newdf,how='inner',on='RegionName')
    univ_prices = univ_prices.dropna()
    
    non_univ_prices=pd.merge(ut,newdf,how='outer',on='RegionName',indicator=True)
    non_univ_prices1=non_univ_prices[non_univ_prices['_merge']=='right_only']
    non_univ_prices1=non_univ_prices1.drop('_merge', axis=1)
    #non_univ_prices1 = non_univ_prices1.dropna()
    
    s, p = ttest_ind(univ_prices['price_ratio'], non_univ_prices1['price_ratio'],nan_policy='omit')
    s2, p2 = ttest_ind(non_univ_prices1['price_ratio'], univ_prices['price_ratio'],nan_policy='omit')
    ans = True, p, "university town"
    return ans
    #return ut
run_ttest()

(True, 6.8182012204080287e-07, 'university town')